# **Content-based Recommenders**

---

## **Overview**
During this exercise you will create and use some content-based profiles for  enerating predictions using Python. It will be necessary to make different computational variations to observe how certain features of computations can bring biases.

***Please upload in Moodle this notebook (executable and in PDF) with the code and answers to the posted questions. ***

## **Instructions**

Before you start just to remind the intuitive algorithm of the content-based filtering:



1.   Computing vectors to describe items (have a dataset of described articles (items)).
2.   Based on the user ratings of items and item’s content profiles, build a user profile preferences
(preferences of attributes).
3. Predicting user interests in items based on a dot product of the item profile vector and user
profile vector. Vectors should be normalized.
4. From this prediction create a top n recommendations or scale for rating-scale predictions



In [37]:
#AUTHOR: Ahonon Gobi Parfait

import numpy as np
import pandas as pd

# read CSV
movies = pd.read_csv('movies.csv')


#Users ratings. 1 means like, -1 dislike, 0 not rated
john_likes=pd.DataFrame([1, -1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0])
joan_likes=pd.DataFrame([-1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1, 0, 0, 0])

#Adding the likes to the Dataset
movies["John_Likes"]=john_likes
movies["Joan_Likes"]=joan_likes

#The dataframe with the dataset
print(movies)

#Copy of the original datased  useful for Task 2
movies_weighted=movies.copy()

# print all together in dataframe by adding the user likes
pd.DataFrame(movies)

                                    Movie  Adventure  Animation  Children  \
0                        Toy Story (1995)          1          0         1   
1                          Jumanji (1995)          0          1         1   
2                 Grumpier Old Men (1995)          0          0         0   
3                Waiting to Exhale (1995)          0          0         1   
4      Father of the Bride Part II (1995)          0          1         0   
5                             Heat (1995)          1          0         0   
6                          Sabrina (1995)          0          0         0   
7                     Tom and Huck (1995)          0          0         1   
8                     Sudden Death (1995)          0          0         0   
9                        GoldenEye (1995)          0          1         0   
10         American President, The (1995)          0          0         1   
11     Dracula: Dead and Loving It (1995)          1          0         0   

,Movie,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Thriller,Action,Horror,Num_Attr,John_Likes,Joan_Likes
0,Toy Story (1995),1,0,1,0,1,1,0,0,0,1,5,1,-1
1,Jumanji (1995),0,1,1,1,0,0,0,1,0,0,4,-1,1
2,Grumpier Old Men (1995),0,0,0,1,1,1,0,0,0,0,3,0,0
3,Waiting to Exhale (1995),0,0,1,1,0,0,1,1,0,0,4,0,1
4,Father of the Bride Part II (1995),0,1,0,0,0,0,0,0,1,1,3,0,0
5,Heat (1995),1,0,0,1,0,0,0,0,0,0,2,1,0
6,Sabrina (1995),0,0,0,0,0,0,0,1,0,1,2,0,0
7,Tom and Huck (1995),0,0,1,1,0,0,1,0,0,1,4,0,0
8,Sudden Death (1995),0,0,0,0,0,1,0,0,1,0,2,0,0
9,GoldenEye (1995),0,1,0,0,1,0,1,0,0,0,3,0,0


### Preparation

For this exercise, the small dataset `movies.csv` will be used. It has twenty (20) entries and each
entry corresponds to the name of a movie, followed by ten values which can be one (1) or zero (0),
where one (1) means that the movie belongs to the genre indicated on the header of the csv file;
the last value represents the number of genres that the movie has (`Num_Attr`)
1. Open the file CBRecommender.py with your favorite Python editor.
2. Read the code following the instructions of this document.
3. Complete the tasks by adding the lines of code that you consider necessary.

**Task 1: Build a recommender with simple unweighted content-based profiles**

Build up a user profile of preferences of movie genre for the user *Joan*. Write the necessary
lines of codes, to this end you may follow these instructions and example:

* Count the total number of positive and negative evaluations associated with each movie genre, and then create a profile with the total score for each genre for user *Joan*.
* For example, *John’s* score for the genre “Adventure” will be equal to three (3) from three (3) positive evaluations of the movies Toy Story, Heat, and casino.
* Basically, you need to use a dot-product-like operation of the user’s ratings dataframe (i.e., john_likes) with the attribute’s vector across the twenty (20) movies. Take as an example of this the following line of code:


In [30]:
#User profile by movie genre
john_likes_score=pd.DataFrame((movies.iloc[:,1:11].values*john_likes.values).sum(axis=0)).T
print(john_likes_score)

   0  1  2  3  4  5  6  7  8  9
0  3 -2 -1  0  0  2 -1 -1  1  0



**Compute the user profile for Joan:**

You could use something similar as the code above to perform the calculations for user *Joan*.


In [31]:
# Count the total number of positive and negative evaluations associated with each movie genre for Joan
joan_likes_score = pd.DataFrame((movies.iloc[:, 1:11].values * joan_likes.values).sum(axis=0)).T

# Display Joan's score for each genre
print("Joan's Score for Each Genre:")
# display in pandas dataframe so it looks nice
print(joan_likes_score)

print("Joan's Score for Each Genre more clearly :")
import pandas as pd
pd.DataFrame(joan_likes_score)

Joan's Score for Each Genre:
   0  1  2  3  4  5  6  7  8  9
0 -2  2  2  3 -1 -2  0  3  0 -1
Joan's Score for Each Genre more clearly :


,0,1,2,3,4,5,6,7,8,9
0,-2,2,2,3,-1,-2,0,3,0,-1


Now, you have the user profile of preferences for each movie genre and also the movie vector with content attributes (the genres of each movie). From this, compute the predicted score for each article for the user *Joan* as explained below and answer the following questions:
1. Produce a dot-product of user’s profile preferences with each movie vector.  
Example for John:
 


In [32]:
#Prediction vales for  user John
pred_john=(movies.iloc[:,1:11].values*john_likes_score.values).sum(axis=1)

movies["Pred_John"]=pred_john

#Showing the prediction scores for John and the names of the movies
print(movies[['Movie','John_Likes','Pred_John']])

                                    Movie  John_Likes  Pred_John
0                        Toy Story (1995)           1          4
1                          Jumanji (1995)          -1         -4
2                 Grumpier Old Men (1995)           0          2
3                Waiting to Exhale (1995)           0         -3
4      Father of the Bride Part II (1995)           0         -1
5                             Heat (1995)           1          3
6                          Sabrina (1995)           0         -1
7                     Tom and Huck (1995)           0         -2
8                     Sudden Death (1995)           0          3
9                        GoldenEye (1995)           0         -3
10         American President, The (1995)           0          0
11     Dracula: Dead and Loving It (1995)           0          4
12                           Balto (1995)           0         -2
13                           Nixon (1995)           0         -2
14                Cutthro

In [33]:
# Write here the code to calculate the prediction vales for  user Joan
#Prediction vales for  user Joan
pred_joan=(movies.iloc[:,1:11].values*joan_likes_score.values).sum(axis=1)

movies["Pred_Joan"]=pred_joan

#Showing the prediction scores for Joan and the names of the movies

print(movies[['Movie','Joan_Likes','Pred_Joan']])



                                    Movie  Joan_Likes  Pred_Joan
0                        Toy Story (1995)          -1         -4
1                          Jumanji (1995)           1         10
2                 Grumpier Old Men (1995)           0          0
3                Waiting to Exhale (1995)           1          8
4      Father of the Bride Part II (1995)           0          1
5                             Heat (1995)           0          1
6                          Sabrina (1995)           0          2
7                     Tom and Huck (1995)           0          4
8                     Sudden Death (1995)           0         -2
9                        GoldenEye (1995)           0          1
10         American President, The (1995)           0          1
11     Dracula: Dead and Loving It (1995)          -1         -4
12                           Balto (1995)           0          7
13                           Nixon (1995)           0          7
14                Cutthro

2.  Which movie does the simple profile predict *John* will like the most, excluding the movies he has already rated?
> *Answer:*  Dracula: Dead and Loving It (1995)

3.  What score does that prediction get?
> *Answer:* 4

4.  How many movies does the model predict the user *Joan* will dislike i.e., have a negative score?
> *Answer:* The model predicts that Joan will dislike 4 movies.

5. Which movie will the user *Joan* like the most, excluding the movies she has already rated?
> *Answer:* The movie that the simple profile predicts Joan will like the most excluding the movies she has already rated is "Balto (1995)".


**Task 2: Build a recommender with simple weighted contentbased
profiles**

It can be noticed that, in the previous computation a movie that has many genres checked could have more influence on the overall user profile than one that had only a few. For instance, the movie Toy Story has five (5) genres, while Sabrina only has two (2).

In this task you will explore whether the previous model (Task 1) may be counting attribute-heavy documents too much resulting in biased prediction scores. For this, write the code that you consider necessary, simply add it under the indicated comment. You may use the dataframe `movies_weighted`, which is a copy of the original dataframe but without the profiles and prediction
scores of the previous task (check the first code block).

1. Build a new weighted model by doing the following:
 

*    Get the square root of the total number of attributes for each movie. You can use the field `Num_Attr` of the dataset for this. 
*   Divide each value of each article vector by the value calculated in the previous point


In [34]:
# Step 1: Calculate the square root of the total number of attributes for each movie
movies_weighted['Sqrt_Num_Attr'] = np.sqrt(movies_weighted['Num_Attr'])

# Step 2: Divide each value of each movie vector by the square root calculated in step 1
for i in range(10):  # Iterate over each attribute column
    column_name = movies_weighted.columns[i + 1]  # Get the name of the column
    movies_weighted[column_name] = movies_weighted[column_name] / movies_weighted['Sqrt_Num_Attr']

# Display the modified DataFrame
# print(movies_weighted.head())

pd.DataFrame(movies_weighted)


,Movie,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Thriller,Action,Horror,Num_Attr,John_Likes,Joan_Likes,Sqrt_Num_Attr
0,Toy Story (1995),0.447214,0.000000,0.447214,0.000000,0.447214,0.447214,0.000000,0.000000,0.000000,0.447214,5,1,-1,2.236068
1,Jumanji (1995),0.000000,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,4,-1,1,2.000000
2,Grumpier Old Men (1995),0.000000,0.000000,0.000000,0.577350,0.577350,0.577350,0.000000,0.000000,0.000000,0.000000,3,0,0,1.732051
3,Waiting to Exhale (1995),0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,4,0,1,2.000000
4,Father of the Bride Part II (1995),0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.577350,0.577350,3,0,0,1.732051
5,Heat (1995),0.707107,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,1,0,1.414214
6,Sabrina (1995),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107,2,0,0,1.414214
7,Tom and Huck (1995),0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.500000,4,0,0,2.000000
8,Sudden Death (1995),0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.707107,0.000000,2,0,0,1.414214
9,GoldenEye (1995),0.000000,0.577350,0.000000,0.000000,0.577350,0.000000,0.577350,0.000000,0.000000,0.000000,3,0,0,1.732051




2.   Compute your new user profiles and new predictions, using the new attribute values and the same schema as in Task 1.



In [35]:
# compute the new user profiles
john_likes_score_weighted = pd.DataFrame((movies_weighted.iloc[:, 1:11].values * john_likes.values).sum(axis=0)).T
joan_likes_score_weighted = pd.DataFrame((movies_weighted.iloc[:, 1:11].values * joan_likes.values).sum(axis=0)).T

# compute the new predictions
pred_john_weighted = (movies_weighted.iloc[:, 1:11].values * john_likes_score_weighted.values).sum(axis=1)
pred_joan_weighted = (movies_weighted.iloc[:, 1:11].values * joan_likes_score_weighted.values).sum(axis=1)

# add the new predictions to the DataFrame
movies_weighted['Pred_John'] = pred_john_weighted
movies_weighted['Pred_Joan'] = pred_joan_weighted

# display the new predictions
print("John's Predictions:")
print(movies_weighted[['Movie', 'John_Likes', 'Pred_John']])

print("Joan's Predictions:")
print(movies_weighted[['Movie', 'Joan_Likes', 'Pred_Joan']])




John's Predictions:
                                    Movie  John_Likes  Pred_John
0                        Toy Story (1995)           1   1.009019
1                          Jumanji (1995)          -1  -0.870053
2                 Grumpier Old Men (1995)           0   0.711105
3                Waiting to Exhale (1995)           0  -0.620053
4      Father of the Bride Part II (1995)           0  -0.213541
5                             Heat (1995)           1   1.370923
6                          Sabrina (1995)           0  -0.353553
7                     Tom and Huck (1995)           0  -0.370053
8                     Sudden Death (1995)           0   1.132724
9                        GoldenEye (1995)           0  -0.805073
10         American President, The (1995)           0   0.044658
11     Dracula: Dead and Loving It (1995)           0   1.333114
12                           Balto (1995)           0  -0.396447
13                           Nixon (1995)           0  -0.331378
14   

3. Have a look at the *John’s* and *Johan’s* prediction scores. Answer the following questions:¨

* Which article does the new model predict John will like the best, excluding the movies he has already rated? Is it different than in Task 1?
> *Answer:* Sudden Death (1995)
   (This differs from Task 1 prediction)

* What score does that prediction get?
> *Answer:* The prediction score for John's favorite movie is: 0.16770554008963812

* How many movies does the new model predict the user Joan will dislike?
> *Answer:* The new model predicts that Joan will dislike 4 movies.

* Which movie will the user Joan like the most, excluding the movies she has already rated? Any changes in comparison to the results of Task 1?
> *Answer:* The movie that the new model predicts Joan will like the most excluding the movies she has already rated is "Four Rooms (1995)".
> *Answer:* (This differs from Task 1 prediction)

* Which model do you consider is better? Under which situation would you use the
unweighted model and under which the weighted model?
> *Answer:* The weighted model is better because it reduces the bias of attribute-heavy documents. The unweighted model may be used when the number of attributes is not a concern, or when the number of attributes is not expected to have a significant impact on the user's preferences.

In [36]:
# 1. Which article does the new model predict John will like the best, excluding the movies he has already rated? Is it different than in Task 1?

# Exclude movies already rated by John
unrated_movies_john = movies_weighted[movies_weighted['John_Likes'] == 0]

# Find the movie with the highest prediction score for John
best_movie_john = unrated_movies_john.loc[unrated_movies_john['Pred_John'].idxmax()]

# Compare with Task 1
best_movie_john_task1 = movies.loc[movies['Pred_John'].idxmax()]

# 2. What score does that prediction get?
prediction_score_john = best_movie_john['Pred_John']

# 3. How many movies does the new model predict the user Joan will dislike?
disliked_movies_joan_weighted = movies_weighted[movies_weighted['Pred_Joan'] < 0]
num_disliked_movies_joan_weighted = len(disliked_movies_joan_weighted)

# 4. Which movie will the user Joan like the most, excluding the movies she has already rated? Any changes in comparison to the results of Task 1?
# Exclude movies already rated by Joan
unrated_movies_joan = movies_weighted[movies_weighted['Joan_Likes'] == 0]

# Find the movie with the highest prediction score for Joan
best_movie_joan = unrated_movies_joan.loc[unrated_movies_joan['Pred_Joan'].idxmax()]

# Compare with Task 1
best_movie_joan_task1 = movies.loc[movies['Pred_Joan'].idxmax()]

# 5. Which model do you consider is better? Under which situation would you use the unweighted model and under which the weighted model?
# We'll compare the performance of the models based on their predictions.

# Print the results
print("1. The new model predicts that John will like the movie:", best_movie_john['Movie'])
print("   (This differs from Task 1 prediction:", best_movie_john_task1['Movie'], ")")
print("2. The prediction score for John's favorite movie is:", prediction_score_john)
print("3. The new model predicts that Joan will dislike", num_disliked_movies_joan_weighted, "movies.")
print("4. The new model predicts that Joan will like the movie:", best_movie_joan['Movie'])
print("   (This differs from Task 1 prediction:", best_movie_joan_task1['Movie'], ")")

1. The new model predicts that John will like the movie: Dracula: Dead and Loving It (1995)
   (This differs from Task 1 prediction: Casino (1995) )
2. The prediction score for John's favorite movie is: 1.3331138468776909
3. The new model predicts that Joan will dislike 4 movies.
4. The new model predicts that Joan will like the movie: Balto (1995)
   (This differs from Task 1 prediction: Jumanji (1995) )
